In [ ]:
import pickle

In [ ]:
import vk_api
vk_session = vk_api.VkApi(token="4ee251bf489b9d88106f08b36239eb0ab39bed07ca3fb9adacf62346cb2981eed59a26f4c29a18123f053")

In [ ]:
vk_session.authorization()

In [ ]:
cached_ids = pickle.load(open("cache.p", "rb"))

In [ ]:
def get_many_of_public(group_link, vk_session):
    global cached_ids
    
    shortname = group_link[group_link.rfind("/")+1:]
    r = vk_session.method("groups.getById", {"group_ids": shortname})
    group_id = r[0]["id"]
    
    if group_id not in cached_ids.keys():
    
        """Все сообщества подписчиков сообщества"""
        users = vk_api.VkTools(vk_session).get_all("groups.getMembers", 100, {'group_id':group_id})['items']
        k = int(len(users) / 1000)
        if k:
            users = users[::k]

        data_for_clustering = []
        users_publicpages = []
        with vk_api.VkRequestsPool(vk_session) as pool:
            
            for i in range(0, len(users), 1000):
                batch = users[i:i+1000]
                data_for_clustering.append(pool.method('users.get', {
                        "user_ids": batch,
                        "fields": "sex,age,education,universities,schools,interests,music,movies,bdate,city,country"
                    }))

                for user_id in batch:
                    users_publicpages.append(pool.method('users.getSubscriptions', {
                                "user_id": user_id
                            }))

        data_for_clustering = [x for x in [x.result for x in data_for_clustering]]
        t = []
        for user in users_publicpages:
            try:
                t.append(user.result['groups']['items'])
            except:
                pass
        users_publicpages = t
        
        wall50 = vk_session.method("wall.get", {"owner_id": -group_id, "filter": "owner", "count": 100})['items'][::2]
        
        result = data_for_clustering, users_publicpages, wall50
        cached_ids[group_id] = result
        pickle.dump(cached_ids, open("cache.p", "wb"))
        return 
    else:
        return cached_ids[group_id]

In [ ]:
from time import time
t = time()
a = get_many_of_public("https://vk.com/goto_msk", vk_session)
print(time() - t)

In [ ]:
len(cached_ids[80270762][1])

In [ ]:
import pickle

In [ ]:
cached_ids = pickle.load(open("cache.p", "rb"))

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne = TSNE()

In [ ]:
cached_ids.keys()

In [ ]:
import pandas as pd
data = pd.read_csv("communities.csv").to_dict()

In [ ]:
indexer = {v: k for k, v in data["link"].items()}
y = {}
for group_link in list(indexer.keys())[1:]:
    shortname = group_link[group_link.rfind("/")+1:]
    if "public" in shortname:
        shortname = shortname[6:]
    y[shortname] = indexer[group_link]
indexer = y

In [ ]:
import vk
session = vk.Session()
api = vk.API(session)

x = {}
for k, v in indexer.items():
    try:
        r = api.groups.getById(group_ids=k)
        group_id = r[0]["gid"]
        print(group_id)
        x[group_id] = v
    except:
        pass
indexer = x

In [ ]:
indexer = pickle.load(open("community_indexer.p", "rb"))

In [ ]:
tsne.fit_transform()